In [7]:
import numpy as np
import pandas as pd


def find_max_f1_weighted(cv_results):
    """Find the maximum weighted F1 score and its corresponding threshold."""
    f1_scores = [
        np.mean(metrics["multiclass_f1_weighted"])
        for metrics in cv_results.values()
        if "multiclass_f1_weighted" in metrics
    ]
    if not f1_scores:
        return 0, None
    max_f1 = max(f1_scores)
    max_threshold = next(
        threshold
        for threshold, metrics in cv_results.items()
        if "multiclass_f1_weighted" in metrics and np.mean(metrics["multiclass_f1_weighted"]) == max_f1
    )
    return max_f1, max_threshold


def process_results(results):
    """Process the results dictionary to extract max F1 scores and thresholds."""
    processed_data = {}

    # Get unique datasets
    datasets = set(dataset for (dataset, _, _, _) in results.keys())

    for dataset in datasets:
        processed_data[dataset] = {}

        # Get unique models for this dataset
        models = set(model for (d, model, _, _) in results.keys() if d == dataset)

        for model in models:
            processed_data[dataset][model] = {}

            # Filter results for current dataset and model
            relevant_results = {key: value for key, value in results.items() if key[0] == dataset and key[1] == model}

            for (_, _, labelling_method, _), data in relevant_results.items():
                # Extract max F1 score and threshold
                max_f1, threshold_at_max = find_max_f1_weighted(data["cv_results"])

                processed_data[dataset][model][labelling_method] = {"max_f1": max_f1, "threshold": threshold_at_max}

    return processed_data


def generate_data_rows(processed_results):
    """Generate data rows for the LaTeX table, grouping models under datasets."""
    labelling_methods = ["knn1", "knn5", "knn5distance", "knn1centroid", "knn1centroid_iqr"]
    model_order = ["EfN-Pretrained", "EfN-Finetuned", "ViT-Pretrained", "ViT-Finetuned"]

    data_rows = []
    for dataset, models in sorted(processed_results.items()):
        if dataset.startswith("Synthetic"):
            continue
        data_rows.append(f"\\multicolumn{{12}}{{l}}{{\\textbf{{{dataset}}}}} \\\\")
        for model in model_order:
            if model in models:
                methods = models[model]
                row = [f"& {model}"]
                row.extend(
                    f" & {methods.get(method, {}).get('threshold', '-'):.2f} & {methods.get(method, {}).get('max_f1', '-'):.2f}"
                    for method in labelling_methods
                )
                data_rows.append(" ".join(row) + " \\\\")
        data_rows.append("\\midrule")

    if data_rows:
        data_rows.pop()  # Remove the last \midrule

    return data_rows


def create_latex_table(data_rows, detail: str = ""):
    """Create the LaTeX table string."""
    latex_table = (
        r"""
\begin{table}[H]
    \centering
    \resizebox{\textwidth}{!}{%
    \begin{tabular}{llcccccccccc}
    \toprule
    & & \multicolumn{2}{c}{knn1} & \multicolumn{2}{c}{knn5} & \multicolumn{2}{c}{knn5distance} & \multicolumn{2}{c}{knn1centroid} & \multicolumn{2}{c}{knn1centroid\_iqr} \\
    \cmidrule(lr){3-4} \cmidrule(lr){5-6} \cmidrule(lr){7-8} \cmidrule(lr){9-10} \cmidrule(lr){11-12}
    & & Thresh & F1 & Thresh & F1 & Thresh & F1 & Thresh & F1 & Thresh & F1 \\
    \midrule
    DATA_ROWS
    \bottomrule
    \end{tabular}%
    }
    \caption{Comparison of Labelling Approaches at the best weighted multiclass F1 score, given a fixed Threshold"""
        + detail
        + r"""}
    \label{tab:recognition-table}
\end{table}
    """
    )
    return latex_table.replace("DATA_ROWS", "\n    ".join(data_rows))


def generate(results, detail=""):
    """Main function to process results and generate the LaTeX table."""
    processed_results = process_results(results)
    data_rows = generate_data_rows(processed_results)
    latex_table = create_latex_table(data_rows, detail)
    return latex_table

In [8]:
import pickle

# filepath = "/workspaces/gorillatracker/sep26_reid_results.pkl"
filepath = "/workspaces/gorillatracker/sep29_reid_results_euclidean.pkl"
with open(filepath, "rb") as f:
    results = pickle.load(f)

latex_table = generate(results, " (Euclidean Distance)")
print(latex_table)


\begin{table}[H]
    \centering
    \resizebox{\textwidth}{!}{%
    \begin{tabular}{llcccccccccc}
    \toprule
    & & \multicolumn{2}{c}{knn1} & \multicolumn{2}{c}{knn5} & \multicolumn{2}{c}{knn5distance} & \multicolumn{2}{c}{knn1centroid} & \multicolumn{2}{c}{knn1centroid\_iqr} \\
    \cmidrule(lr){3-4} \cmidrule(lr){5-6} \cmidrule(lr){7-8} \cmidrule(lr){9-10} \cmidrule(lr){11-12}
    & & Thresh & F1 & Thresh & F1 & Thresh & F1 & Thresh & F1 & Thresh & F1 \\
    \midrule
    \multicolumn{12}{l}{\textbf{Bristol}} \\
    & EfN-Pretrained  & 2.66 & 0.52  & 2.66 & 0.50  & 2.78 & 0.48  & 2.78 & 0.42  & 4.95 & 0.33 \\
    & EfN-Finetuned  & 11.54 & 0.46  & 11.54 & 0.45  & 12.90 & 0.44  & 11.20 & 0.39  & 10.18 & 0.38 \\
    & ViT-Pretrained  & 21.84 & 0.62  & 21.22 & 0.61  & 21.22 & 0.58  & 19.97 & 0.51  & 21.84 & 0.44 \\
    & ViT-Finetuned  & 11.85 & 0.68  & 11.85 & 0.68  & 12.54 & 0.64  & 12.54 & 0.53  & 11.50 & 0.49 \\
    \midrule
    \multicolumn{12}{l}{\textbf{Bristol+min25+max25}} 

In [9]:
filepath = "/workspaces/gorillatracker/sep29_reid_results_cosine.pkl"
with open(filepath, "rb") as f:
    results = pickle.load(f)

latex_table = generate(results, " (Cosine Similarity)")
print(latex_table)


\begin{table}[H]
    \centering
    \resizebox{\textwidth}{!}{%
    \begin{tabular}{llcccccccccc}
    \toprule
    & & \multicolumn{2}{c}{knn1} & \multicolumn{2}{c}{knn5} & \multicolumn{2}{c}{knn5distance} & \multicolumn{2}{c}{knn1centroid} & \multicolumn{2}{c}{knn1centroid\_iqr} \\
    \cmidrule(lr){3-4} \cmidrule(lr){5-6} \cmidrule(lr){7-8} \cmidrule(lr){9-10} \cmidrule(lr){11-12}
    & & Thresh & F1 & Thresh & F1 & Thresh & F1 & Thresh & F1 & Thresh & F1 \\
    \midrule
    \multicolumn{12}{l}{\textbf{Bristol}} \\
    & EfN-Pretrained  & 0.85 & 0.51  & 0.85 & 0.49  & 0.85 & 0.49  & 0.85 & 0.42  & 0.12 & 0.33 \\
    & EfN-Finetuned  & 0.68 & 0.45  & 0.68 & 0.44  & 0.68 & 0.43  & 0.68 & 0.38  & 0.68 & 0.36 \\
    & ViT-Pretrained  & 1.25 & 0.33  & 1.25 & 0.33  & 1.25 & 0.33  & 1.25 & 0.33  & 1.25 & 0.33 \\
    & ViT-Finetuned  & 0.70 & 0.66  & 0.70 & 0.66  & 0.70 & 0.63  & 0.70 & 0.55  & 0.70 & 0.51 \\
    \midrule
    \multicolumn{12}{l}{\textbf{Bristol+min25+max25}} \\
    & EfN-Pr